# EXPLOTACION DE POZOS DE PETROLEO: OILYGIATN

### Objetivo

* Entrenar el modelo de regresion lineal y obtener los mejores 200 pozo para la inversion de la Empresa OilyGiant

## Carga y Exploracion de los datos

In [ ]:
#Importacion de librerias

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
#Uso root_mean_squared_error porque la version que uso mean_squared_error esta 'deprecated'
from sklearn.metrics import root_mean_squared_error

In [ ]:
#Carga de los datos

data_0 = pd.read_csv('geo_data_0.csv')
data_1 = pd.read_csv('geo_data_1.csv')
data_2 = pd.read_csv('geo_data_2.csv')

#Verificando la estructura de los datos
print("Data 0")
data_0.info()
print("\nData 01")
data_1.info()




Data 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

Data 01
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

Data 02
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns)

In [22]:
print("Data 02")
data_2.info()

Data 02
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [25]:
data_0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [ ]:
#Verificamos rapidamente los valores duplicados:
print("Datos duplicados data 0:", data_0.duplicated().sum())

print("\nDatos duplicados data 1:", data_1.duplicated().sum())

print("\nDatos duplicados data 2:", data_2.duplicated().sum())


Datos duplicados data 0: 0

Datos duplicados data 1: 0

Datos duplicados data 2: 0


* Hacemos un analisis premilinar de los datos, para verificar si tienen datos nulos, datos duplicados y si el tipo de dato es el correcto.

## Prepraracion de los Datos

In [27]:
#Creamos una funcion para dividir, entrenar y evaluar el modelo:
def train_and_evaluate(data):
    features = data.drop(['id', 'product'], axis = 1)
    target = data['product']
    
    #Division de los datos:
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, random_state=12345, test_size=0.25)
    
    #Entrenamiento del Modelo:
    model = LinearRegression()
    model.fit(features_train, target_train)
    
    #Predicciones y calculo del RSME
    predictions = model.predict(features_valid)
    rmse = root_mean_squared_error(target_valid, predictions)
    
    mean_reserves = predictions.mean()
    
    return predictions, target_valid.reset_index(drop=True), mean_reserves, rmse

* Creamos la funcion para hacer la division de los datos con la consigna del 75% y 25% para datos de entrenamiento y de prueba respectivamente, entrenamos el modelo, y devolvemos las predicciones, los datos de validacion, el promedio de las resevas y el RMSE

In [36]:
# Entrenar y evaluar para cada region:
results_0 = train_and_evaluate(data_0)
results_1 = train_and_evaluate(data_1)
results_2 = train_and_evaluate(data_2)

#Imprimir los resultados:
for i, results in enumerate([results_0, results_1, results_2]):
    print(f'Region {i}: RSME= {results[3]:.2f}, Media de Reservas = {results[2]:.2f}')

Region 0: RSME= 37.58, Media de Reservas = 92.59
Region 1: RSME= 0.89, Media de Reservas = 68.73
Region 2: RSME= 40.03, Media de Reservas = 94.97


* Tenemos la Region 0 y 2, con un rmse de valor alto, esto indica incertidumbre en las predicciones que puede llevar a mas errores, pero la media de reservas es mayor que en la region 1, lo que puede generar mas rendimiento que la region 1, apesar de tener una menor incertidumbre.

## Calculo de ganancias

In [ ]:
#Parametros:
budget = 100e6 #Inversion
unit_profit = 4500  #Ingreso por un barril en USD
wells_selected = 200 # Los 200 mejores puntos
min_profit_threshold = 111.1 #Monto minimo de unidades de barril para no tener perdidas

#Funcion para el calculo de la ganancia estimada de la region:
def calculate_profit(predictions):
    top_predictions = np.sort(predictions)[-wells_selected]
    total_reserves = top_predictions.sum()
    revenue = total_reserves * unit_profit
    
    return revenue - budget



* Definimos los parametros mencionados en el proyecto, y generamos la funcion para el calculo de la ganancia estimada, se colocar el np.sort porque la devolucion devolucion de nuestra funcion anteriror no es un dataframe, y el signo '-' indica que tomaremos los 200 ultimos que serian las mayores predicciones. Para que finalmente calculemos todas las reservas de cada region, y se reste con la inversion.

In [41]:
# Calcular las ganancias para cada region:
profit_0 = calculate_profit(results_0[0])
profit_1 = calculate_profit(results_1[0])
profit_2 = calculate_profit(results_2[0])

print(f"Ganancia Region 0: ${profit_0:,.2f}")
print(f"Ganancia Region 1: ${profit_1:,.2f}")
print(f"Ganancia Region 2: ${profit_2:,.2f}")

Ganancia Region 0: $-99,332,353.27
Ganancia Region 1: $-99,377,151.86
Ganancia Region 2: $-99,359,160.17


* Verificando los resultados nos encontramos que ninguna region logra generar ganancias, lo que nos indica que las reservas de los 200 mejores pozos no es suficiente para cubrir el presupuesto de los 100 millones de USD.

## Analisis de riesgo y bootstrapping

In [31]:
#Funcion para aplicar bootstrapping y calcular el riesgo de perdida:
def bootstrap_risk(predictions, confidence_level=0.025, n_samples=1000):
    profits = []
    
    for _ in range(n_samples):
        sample = np.random.choice(predictions, size=wells_selected, replace=True)
        revenue = sample.sum() * unit_profit - budget
        profits.append(revenue)
    
    # Calcular estadísticos
    profits = np.array(profits)
    mean_profit = profits.mean()
    lower_bound = np.percentile(profits, confidence_level * 100)
    upper_bound = np.percentile(profits, (1 - confidence_level) * 100)
    risk = np.mean(profits < 0) * 100  # porcentaje de riesgo de pérdida
    
    return mean_profit, (lower_bound, upper_bound), risk

* Aplicamos la tecnica de bootstrapping con la generacion de 1000 muestras, para cuantificar el benificio esperado como tambien el riesgo de perdida en funcion de las reservas estimadas, si el riesgo esta por debajo del 2.5% la ganancia promedio es positiva y la region puede considerarse una buena opcion de inversion.

In [32]:
#Evaluar el reisgo de cada region:
risk_0 = bootstrap_risk(results_0[0])
risk_1 = bootstrap_risk(results_1[0])
risk_2 = bootstrap_risk(results_2[0])

for i, risk in enumerate([risk_0, risk_1, risk_2]):
    print(f"Region {i}: Beneficio Promedio = ${risk[0]:,.2f}, Intervalo de confianza 95% = {risk[1]}, Riesgo de perdida = {risk[2]:.2f}%")

Region 0: Beneficio Promedio = $-16,698,039.05, Intervalo de confianza 95% = (-19523776.65822784, -13890982.073329234), Riesgo de perdida = 100.00%
Region 1: Beneficio Promedio = $-38,166,612.79, Intervalo de confianza 95% = (-43943949.118129805, -32327584.485645182), Riesgo de perdida = 100.00%
Region 2: Beneficio Promedio = $-14,495,814.84, Intervalo de confianza 95% = (-16911188.08313878, -11839545.832762577), Riesgo de perdida = 100.00%


* Tenemos que para cada region el beneficio es negativo y para el intervalo de confianza del 95% tenemos un riesgo de perdida del 100% para cada region.

## Conclusiones

1.  Los resultados obtenidfos indican que, segun las predicciones actuales, abrir nuevos pozos en cualquiera de las 3 regiones resulta en perdidas significativas. Bajo la condicion de 100 millones de dolares no justifca los ingresos potenciales de la reservas.

2. Es posible que el uso de la regresion lineal como modelo no sea lo suficientemente robusto para la prediccion del volumen.

3. El riesgo es del 100% en todas las regiones, lo que indica una mala inversiones puesto que no sera recuperada.